In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel,  TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import optuna
from transformers import EarlyStoppingCallback
from sklearn.model_selection import train_test_split
import json

In [13]:
BOOKS_PATH = "processed_books.txt"

In [14]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [15]:
def load_and_split_data(file_path, test_size=0.1):
    with open(file_path, "r") as file:
        text = file.read()
    
    stories = text.split("\n\n")
    train_stories, test_stories = train_test_split(stories, test_size=test_size, random_state=42)

    return train_stories, test_stories

In [16]:
train_stories, test_stories = load_and_split_data(BOOKS_PATH)

In [17]:
train_file_path = "train_dataset.txt"
test_file_path = "test_dataset.txt"

with open(train_file_path, "w") as file:
    file.write("\n\n".join(train_stories))
with open(test_file_path, "w") as file:
    file.write("\n\n".join(test_stories))

In [18]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_file_path,
    block_size=128
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_file_path,
    block_size=128
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False # If set to False, the labels are the same as the inputs with the padding tokens ignored
)

In [19]:
def objective(trial, train_dataset,test_dataset, data_collator):

    # Define hyperparameters using trial.suggest_* methods
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [4, 8, 16])
    weight_decay = trial.suggest_float("weight_decay", 0, 0.1)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 500)
    epochs = 3

    # Set up the training arguments using the suggested hyperparameters
    training_args = TrainingArguments(
        output_dir="./output",
        overwrite_output_dir=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        evaluation_strategy="steps",
        logging_dir="./logs",
        logging_steps=500,
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        report_to=["tensorboard"],
        seed=42,
        disable_tqdm=True,
    )

    # Set up the Trainer instance using the suggested hyperparameters
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Train the model and return the best loss
    trainer.train()
    best_loss = trainer.evaluate()["eval_loss"]
    return best_loss

In [20]:
# Take only 1% of the train and test datasets so it's faster to hyper parameter tune
train_dataset_hpt = train_dataset[:int(len(train_dataset) * 0.01)]
eval_dataset_hpt= test_dataset[:int(len(test_dataset) * 0.01)]

print("Training dataset size:", len(train_dataset_hpt))
print("Evaluation dataset size:", len(eval_dataset_hpt))

Training dataset size: 1020
Evaluation dataset size: 98


In [21]:
# Create an Optuna study
study = optuna.create_study(direction="minimize", study_name="gpt2_hyperparameter_tuning")

# Optimize the hyperparameters
study.optimize(lambda trial: objective(trial, train_dataset_hpt, eval_dataset_hpt, data_collator), n_trials=20, timeout=3600)

[I 2023-04-17 17:32:51,746] A new study created in memory with name: gpt2_hyperparameter_tuning
C:\Users\reidp\AppData\Local\Temp\ipykernel_18476\491586242.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
c:\Users\reidp\miniconda3\envs\torch_gpu\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'train_runtime': 67.7811, 'train_samples_per_second': 45.145, 'train_steps_per_second': 5.665, 'train_loss': 4.422989209493001, 'epoch': 3.0}


[I 2023-04-17 17:34:00,551] Trial 0 finished with value: 6.1424336433410645 and parameters: {'learning_rate': 2.419197584455757e-05, 'per_device_train_batch_size': 8, 'weight_decay': 0.08044739840132131, 'warmup_steps': 229}. Best is trial 0 with value: 6.1424336433410645.


{'eval_loss': 6.1424336433410645, 'eval_runtime': 0.5595, 'eval_samples_per_second': 175.154, 'eval_steps_per_second': 23.235, 'epoch': 3.0}
{'loss': 4.0706, 'learning_rate': 1.0999455852539922e-05, 'epoch': 1.96}
{'eval_loss': 6.32277250289917, 'eval_runtime': 0.5505, 'eval_samples_per_second': 178.02, 'eval_steps_per_second': 23.615, 'epoch': 1.96}
{'train_runtime': 88.1986, 'train_samples_per_second': 34.694, 'train_steps_per_second': 8.674, 'train_loss': 4.037008527369281, 'epoch': 3.0}


[I 2023-04-17 17:35:29,326] Trial 1 finished with value: 6.32277250289917 and parameters: {'learning_rate': 1.1165485374842412e-05, 'per_device_train_batch_size': 4, 'weight_decay': 0.02402148423109477, 'warmup_steps': 496}. Best is trial 0 with value: 6.1424336433410645.


...

{'eval_loss': 13.622440338134766, 'eval_runtime': 0.5735, 'eval_samples_per_second': 170.874, 'eval_steps_per_second': 22.667, 'epoch': 3.0}


C:\Users\reidp\AppData\Local\Temp\ipykernel_18476\491586242.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
c:\Users\reidp\miniconda3\envs\torch_gpu\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'train_runtime': 54.6698, 'train_samples_per_second': 55.972, 'train_steps_per_second': 3.512, 'train_loss': 0.09289287527402242, 'epoch': 3.0}


[I 2023-04-17 17:57:38,566] Trial 19 finished with value: 13.563915252685547 and parameters: {'learning_rate': 4.591045607150248e-05, 'per_device_train_batch_size': 16, 'weight_decay': 0.06072850400850554, 'warmup_steps': 47}. Best is trial 0 with value: 6.1424336433410645.


{'eval_loss': 13.563915252685547, 'eval_runtime': 0.6551, 'eval_samples_per_second': 149.595, 'eval_steps_per_second': 19.844, 'epoch': 3.0}


In [24]:
best_params = study.best_params

In [27]:
# Save the best hyperparameters to a JSON file
with open("best_hyperparams.json", "w") as f:
    json.dump(best_params, f)